In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import json
import numpy as np
from functions import create_next_model_folder,clean_author

In [2]:
MODEL_PATH = ["mixedbread-ai/mxbai-embed-large-v1"]
DATA_PATH = ['../Data/AMC/aime.json',\
               '../Data/AMC/amc12.json']
OUT_PATH = '../Data/Embeddings'

In [3]:
get_name = lambda x: x.split('/')[-1]

In [9]:
for model_name in MODEL_PATH:
    model = SentenceTransformer(model_name)
    for data_path in DATA_PATH:
        with open(data_path, 'r') as file:
            data = json.load(file)
        output_questions = []
        output_context = []
        for input_dict in (data):
            question = input_dict.get('problem', '')
            solution_list = input_dict.get('solutions', [])  # Default to an empty list if not found
            solution = min(solution_list, key=lambda x:len(x['solution']))
            output_questions.append(question)
            output_context.append(f"{question} Solution: {clean_author(solution['solution'])}")
        embeddings = model.encode(output_questions)
        embed_folder = create_next_model_folder(OUT_PATH)
        np.save(embed_folder + '/' + get_name(model_name) + "::" + get_name(data_path) +'.npy', embeddings)
        with open(embed_folder + '/' + 'context.json', 'w') as f:
            json.dump(output_context, f)